In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from tqdm import tqdm
import metnum
import numpy
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score,precision_score,recall_score
import csv

In [2]:
def dividir_en_training_testing(M,percentage):
    limit = int(percentage * M.shape[0]) 
    M_train = M[:limit]
    M_val = M[limit:]
    return M_train,M_val


In [3]:
def dividir_en_X_y(M):
    X = M[M.columns[1:]].values
    y = M["label"].values.reshape(-1, 1)
    return X,y

In [4]:
def cargar_datos_de_experimento(percentage,cant_muestra=42000):
    df_train = pd.read_csv("../data/train.csv")
    df_train = shuffle(df_train)
    df_train = df_train[:cant_muestra]
    X,y = dividir_en_X_y(df_train)
    X_train,X_val = dividir_en_training_testing(X,percentage)
    y_train,y_val = dividir_en_training_testing(y,percentage)
    return X_train,X_val,y_train,y_val
    

In [9]:
def correr_Knn_con_todos_los_k_posibles(porcentage_para_entrenar,cant_muestras=42000):
    X_train,X_val,y_train,y_val = cargar_datos_de_experimento(porcentage_para_entrenar,cant_muestras)
    file = open("../resultados/knn_solo_resultados.txt", "w")
    
    accuracy_of_all_k = []
    precision_df = pd.DataFrame(columns=(0,1,2,3,4,5,6,7,8,9))
    recall_df = pd.DataFrame(columns=(0,1,2,3,4,5,6,7,8,9))
    
    # Hago el fit generico que guarda los datos 
    clf = metnum.KNNClassifier(1)
    clf.fit(X_train, y_train)
    
    for k in tqdm(range(1,X_train.shape[0]+1,500)):
        
        # Correr knn 
        clf = metnum.KNNClassifier(k)
        y_pred = clf.predict(X_val)
        
        # Calcular metricas de interes
        labels= [0,1,2,3,4,5,6,7,8,9]
        precision = precision_score(y_val, y_pred,labels=labels, average=None)
        accuracy =  accuracy_score(y_val, y_pred)
        recall   =  recall_score(y_val,y_pred,labels=labels, average=None)
        
        # Escribir los resultados
        accuracy_of_all_k.append(accuracy)
    
    
            # Agregar una fila al dataframe de precision
        digit = 0 
        precision_dict={}
        for i in range(0,10,1):
            precision_dict[digit]=precision[i]
            digit += 1 
        precision_df = precision_df.append(precision_dict,ignore_index=True)

             # Agregar una fila al dataframe de recall
        digit = 0 
        recall_dict={}
        for i in range(0,10,1):
            recall_dict[digit]=recall[i]
            digit += 1 
        recall_df = recall_df.append(recall_dict,ignore_index=True)
 
    # Escribo los resultados a un archivo para no tener que correr devuelta los resultados.
    precision_df.to_csv('knn_solo_precision.csv', index=False) 
    recall_df.to_csv('knn_solo_recall.csv', index=False)   

    with open('knn_solo_acuracy.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(accuracy_of_all_k)

    file.close()

    

In [ ]:
correr_Knn_con_todos_los_k_posibles(0.8)

  0%|          | 0/68 [00:00<?, ?it/s]